批量处理

1. try some tokenizer

In [55]:
import pandas as pd
import re
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
def tokenize(log_content):
    words = log_content.split()
    words = [word for word in words if not re.search(r'\d', word)]
    return words


def vectorize(tokenized_logs):
    vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
    return vectorizer.fit_transform(tokenized_logs)


def cluster(vectorized_logs, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(vectorized_logs)
    return kmeans.labels_


# 读取CSV文件
dataset = 'Spark'
df = pd.read_csv(
    f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')

# 选择某一列，例如'column_name'
column = df['Content']

# 将该列转换为列表
column_list = column.tolist()

tokenized_logs = [tokenize(content) for content in column_list]


labels = cluster(vectorize(tokenized_logs), 40)

d:\Develop\anaconda\envs\langchain38\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\Develop\anaconda\envs\langchain38\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Develop\anaconda\envs\langchain38\lib\site-packages\sklearn\base.py:1151: ConvergenceWarning: Number of distinct clusters (38) found smaller than n_clusters (40). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [57]:
num = 0
for i, l in enumerate(labels):
    if l == 1:  # 12, 14 , 17, 29是异常的模板
        print(column_list[i])
        num += 1
print(num)

Times: total = 1072, boot = 856, init = 210, finish = 6
Times: total = 1114, boot = 885, init = 223, finish = 6
Times: total = 1074, boot = 869, init = 199, finish = 6
Times: total = 1078, boot = 851, init = 219, finish = 8
Times: total = 1077, boot = 865, init = 206, finish = 6
Times: total = 38, boot = 11, init = 27, finish = 0
Times: total = 42, boot = 12, init = 30, finish = 0
Times: total = 41, boot = 15, init = 26, finish = 0
Times: total = 40, boot = 7, init = 33, finish = 0
Times: total = 42, boot = 13, init = 28, finish = 1
Times: total = 42, boot = -4131, init = 4172, finish = 1
Times: total = 41, boot = -4122, init = 4162, finish = 1
Times: total = 42, boot = -4121, init = 4162, finish = 1
Times: total = 42, boot = -4138, init = 4179, finish = 1
Times: total = 42, boot = -4134, init = 4175, finish = 1
Times: total = 39, boot = -147, init = 186, finish = 0
Times: total = 39, boot = -147, init = 186, finish = 0
Times: total = 41, boot = -139, init = 179, finish = 1
Times: tota

In [2]:

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

# 读取CSV文件
dataset = 'Proxifier'
df = pd.read_csv(
    f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')

# 选择某一列，例如'column_name'
column = df['Content']

# 将该列转换为列表
column_list = column.tolist()

# 特征提取
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(column_list)

n = 16
# 聚类
kmeans = KMeans(n_clusters=n)  # n是模板的数量
kmeans.fit(X)

# 输出每个log message的分组结果
labels = kmeans.labels_

d:\Develop\anaconda\envs\langchain38\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


显示每一批分组情况

In [3]:
num = 0
for i,l in enumerate(labels):
    if l == 3:  # 12, 14 , 17, 29是异常的模板
        print(column_list[i])
        num +=1
print(num)

formi.baidu.com:843 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403
127.0.0.1:135 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403
127.0.0.1:135 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403
203.205.129.102:8080 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403
mtalk.google.com:5228 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403
update.hicloud.com:8180 error : Could not connect through proxy proxy.cse.cuhk.edu.hk:5070 - Proxy server cannot establish a connection with the target, status code 403
local-p2p.qq.co

通过三个参数选取最合适的n值

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd

# 读取CSV文件
dataset = 'Proxifier'
df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')

# 选择某一列，例如'column_name'
column = df['Content']

# 将该列转换为列表
column_list = column.tolist()

# 特征提取
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(column_list)

# 聚类误差
errors = []

# 轮廓系数
silhouettes = []

# Calinski-Harabasz指数
calinski_harabasz_scores = []

# 模板数量的范围
range_n_clusters = range(2, 20)

for n_clusters in range_n_clusters:
    # 聚类
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X)

    # 计算聚类误差
    error = kmeans.inertia_
    errors.append(error)

    # 计算轮廓系数
    silhouette = silhouette_score(X, kmeans.labels_)
    silhouettes.append(silhouette)

    # 计算Calinski-Harabasz指数
    calinski_harabasz = calinski_harabasz_score(X.toarray(), kmeans.labels_)
    calinski_harabasz_scores.append(calinski_harabasz)

# 绘制聚类误差图
plt.figure()
plt.plot(range_n_clusters, errors, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Cluster error')
plt.title('The Elbow Method')
plt.show()

# 绘制轮廓系数图
plt.figure()
plt.plot(range_n_clusters, silhouettes, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('The Silhouette Method')
plt.show()

# 绘制Calinski-Harabasz指数图
plt.figure()
plt.plot(range_n_clusters, calinski_harabasz_scores, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Calinski-Harabasz Index')
plt.title('The Calinski-Harabasz Method')
plt.show()